# Чтение и запись данных. Часть 2

## 1. Работа с базой данных из pandas


### 1.1 

Подготовка данных:

In [1]:
import pip
pip.main(['install', 'psycopg2'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
import psycopg2
import pandas as pd
import sqlite3

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()

# получаем имена таблиц из базы
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

print("Какие таблицы содержатся в Postgres: %s" % tables_data)

df = pd.read_sql_query("select * from public.ratings limit 10;", conn)
df.head()

Какие таблицы содержатся в Postgres: [('keywords',), ('links',), ('ratings',), ('exploratory',), ('course_purchases',), ('joi',)]


,userid,movieid,rating,timestamp
0,1888,2881,3.5,1055312882
1,1888,2901,3.5,1055312386
2,1888,2912,4.0,1092375648
3,1888,2918,4.0,1151576329
4,1888,2944,3.5,1055313087


**Задание простого уровня** <br>Посчитайте количество записей в каждой из таблиц **keywords, links, ratings** и выведите в формате имя *таблицы:количество строк*

*Результат работы:*
<pre>
table keywords:92838
table links:91686
table ratings:1555552
</pre>

In [3]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

table_names = [i[0] for i in tables_data][0:3]
for table in table_names:
        sql_str = "SELECT COUNT(*) FROM public.{};".format(table)
        cursor.execute(sql_str)
        row_count = [a for a in cursor.fetchall()][0][0]
        print("table {}:{}".format(table, row_count))

cursor.close()

table keywords:92838
table links:91686
table ratings:1555552


### 1.2

**Задание среднего уровня** <br>Напечатайте колонки таблицы *ratings*. Подсказка: поисследуйте объект **cursor** с помощью встроенной в python команды *dir()*

In [4]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = "SELECT * FROM ratings LIMIT 0;"
cursor.execute(sql_str)
conn.commit()

table_rating_fields = [field[0] for field in cursor.description]
print("Поля таблицы ratings: %s" % table_rating_fields)

cursor.close()

Поля таблицы ratings: ['userid', 'movieid', 'rating', 'timestamp']


### 1.3

**Задание высокого уровня**
* Посмотрите на столбцы таблицы **links** и столбцы таблицы **ratings** по какому полю можно заджойнить эти две таблицы
* Посчитайте количество фильмов из links, у который нет оценок в ratings.

In [5]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = "SELECT * FROM links LIMIT 0;"
cursor.execute(sql_str)
conn.commit()

table_link_fields = [field[0] for field in cursor.description]
print("Поля таблицы links: %s" % table_link_fields)

cursor.close()

Поля таблицы links: ['movieid', 'imdbid', 'tmdbid']


Таблицы links и ratings можно заджойнить по полю movieid.

In [6]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()


sql_str = """
    SELECT COUNT(DISTINCT links.movieid)
    FROM links
    LEFT JOIN ratings ON links.movieid = ratings.movieid 
    WHERE ratings.rating is NULL;
"""
# Еще один вариант, который работает, если все значения rating в таблице ratings не пустые, имеющие рейтинг
# sql_str = """
#     SELECT (COUNT(DISTINCT links.movieid) - COUNT(DISTINCT ratings.movieid))
#     FROM links
#     LEFT JOIN ratings ON links.movieid = ratings.movieid;
# """

cursor.execute(sql_str)
row_count = [a for a in cursor.fetchall()][0][0]
conn.commit()

print("Количество фильмов без рейтингов: %s" % row_count)

cursor.close()

Количество фильмов без рейтингов: 28333


## 2. Работа с БД: MongoDB

### 2.1

Подготовка данных

In [7]:
import pip
pip.main(['install', 'pymongo'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [8]:
from pymongo import MongoClient

mongo_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 27017,
    "user": "students",
    "password": "X63673t47Gl03Sq",
    "authSource": "movies"
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]

print("Коллекции, доступные в MongoDB: %s" % db.list_collection_names())

collection = db['tags']
print("Число документов в коллекции %s" % collection.estimated_document_count())

Коллекции, доступные в MongoDB: ['tags', 'users']
Число документов в коллекции 158680


**Задание простого уровня** <br>Посчитайте количество контента, у которого встречается тэг "toy"

* сделайте выборку такого контента (а точнее - поля `id`) в питоновский `list`
* подсчитайте количество элементов в `list`

In [9]:
selector = {'name': 'toy'}
exclude_id = {'_id': False}

mongo_cursor = collection.find(projection=exclude_id, filter = selector)

cursor_items = set([mongo_cursor])

print("Количество УНИКАЛЬНОГО контента с тэгом 'toy': %s\n" % len(cursor_items))

list_items = [item for item in mongo_cursor]
count_elements = len(list_items)
print("Список элементов с тэгом 'toy': %s\n" % list_items)
print("Количество найденных элементов с тэгом 'toy': %s\n" % count_elements)

Количество УНИКАЛЬНОГО контента с тэгом 'toy': 1

Список элементов с тэгом 'toy': [{'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}]

Количество найденных элементов с тэгом 'toy': 19



### 2.2

**Задание среднего уровня** <br> Выведите **только названия** 10 самых непопулярных тэгов <br>

Подсказка: посмотрите в уроке, как MongoDB позволяет также выполнять сложные агрегирующие запросы средствами СУБД

*Результат работы*
<pre>
Счётчик тэгов ['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']
</pre>

In [10]:
pipline = [
    {"$group":
        {"_id": "$name",
         "tag_count":
            {"$sum": 1}
        }
    },
    
    {"$sort":
        {"tag_count": 1}
     },
    {"$project":
        {"_id": 1}
    },
    {"$limit": 10}
    
]

tags = [i for i in collection.aggregate(pipline)]
tags_list = []
for item in tags:
    for key in item.keys():
        tags_list.append(item[key])

print("Счетчик тегов", tags_list)

Счетчик тегов ['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']


### 2.3

**Задание высокого уровня**

1. Подключитесь к Postgres. Найдите top-10 самого популярного контента
1. Сохраните в файл `content_popularity.json` в виде
<pre>
{'movieId': 931, 'popularity': 999}
{'movieId': 429, 'popularity': 111}
</pre>

Этот формат называется single-line json и его можно загрузить в MongoDB с помощью команды
<pre>
mongoimport --host $APP_MONGO_HOST --port $APP_MONGO_PORT --db movies --collection tags --file content_popularity.json
</pre>

**ВНИМАНИЕ!** Это просто пример как можно использовать single-line JSON, заливать его никуда не нужно - в задаче требуется только подготовить файл в правильном виде

*Подсказка* воспользуйтесь функцией `json.dumps`

*Результат работы* - файл `content_popularity.json`:

<pre>
{"movieId": 318, "popularity": 5626}
{"movieId": 356, "popularity": 5464}
{"movieId": 296, "popularity": 5242}
{"movieId": 593, "popularity": 5048}
{"movieId": 2571, "popularity": 4718}
{"movieId": 260, "popularity": 4556}
{"movieId": 480, "popularity": 4540}
{"movieId": 527, "popularity": 4120}
{"movieId": 1, "popularity": 4072}
{"movieId": 110, "popularity": 3928}
</pre>

In [69]:
import json
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()


sql_str = """
    SELECT movieid, rating FROM ratings ORDER BY rating DESC LIMIT 10;
"""

sql_str = """
    SELECT movieid, COUNT(*) as popularity
    FROM public.ratings 
    GROUP BY movieid
    ORDER BY popularity DESC
    LIMIT 10;
"""
cursor.execute(sql_str)

with open('content_popularity.json', 'w') as f:
    for row in cursor:
        to_json = {'movieid': row[0], 'popularity': row[1]}
        f.write(json.dumps(to_json) + '\n')

conn.close()

In [70]:
with open('content_popularity.json') as f_json:
    print(f_json.read())

{"movieid": 318, "popularity": 5626}
{"movieid": 356, "popularity": 5464}
{"movieid": 296, "popularity": 5242}
{"movieid": 593, "popularity": 5048}
{"movieid": 2571, "popularity": 4718}
{"movieid": 260, "popularity": 4556}
{"movieid": 480, "popularity": 4540}
{"movieid": 527, "popularity": 4120}
{"movieid": 1, "popularity": 4072}
{"movieid": 110, "popularity": 3928}

